In [1]:
import pandas as pd
import utils
import model

In [2]:
stmt = model.session.execute("""
    select
        idinspection,
        numero_inspection,
        synthese_eval,

        CASE 
            WHEN numero_agrement is null  THEN ''
            ELSE numero_agrement
        END	num_agrement,

        siret,
        libelle_etablissement,
        adresse,
        code_postal,
        commune_norm,
        libelle_activite
    from inspection ins
    join etablissement eta on ins.idetablissement = eta.idetablissement
    join activite act on ins.idactivite = act.idactivite
""")

commitCount = 0

for row in stmt:
    idinspection = int(row[0])
    toSplit = utils.normalisation_nom_ville(str(row[2]).strip()) + ' ' + utils.normalisation_nom_ville(str(row[3]).strip()) + ' ' + utils.normalisation_nom_ville(str(row[4]).strip()) + ' ' + utils.normalisation_nom_ville(str(row[5]).strip()) + ' ' + utils.normalisation_nom_ville(str(row[6]).strip()) + ' ' + utils.normalisation_nom_ville(str(row[7]).strip()) + ' ' + utils.normalisation_nom_ville(str(row[8]).strip()) + ' ' + utils.normalisation_nom_ville(str(row[9]).strip())
    toSplit = toSplit.replace('.', ' ')
    toSplit = toSplit.replace(',',' ')
    toSplit = toSplit.replace('/',' ')
    toSplit = toSplit.replace('!',' ')
    toSplit = toSplit.replace('"',' ')
    toSplit = toSplit.replace('#',' ')
    toSplit = toSplit.replace('%',' ')
    toSplit = toSplit.replace('&',' ')
    toSplit = toSplit.replace('(',' ')
    toSplit = toSplit.replace(')',' ')
    toSplit = toSplit.replace('*',' ')
    toSplit = toSplit.replace('+',' ')
    toSplit = toSplit.replace(':',' ')
    toSplit = toSplit.replace('?',' ')
    toSplit = toSplit.replace('@',' ')
    toSplit = toSplit.replace('[',' ')
    toSplit = toSplit.replace(']',' ')
    toSplit = toSplit.replace('_',' ')
    toSplit = toSplit.replace('|',' ')
    toSplit = toSplit.replace('\t',' ')
    toSplit = toSplit.replace('    ',' ')
    toSplit = toSplit.replace('   ',' ')
    toSplit = toSplit.replace('  ',' ')

    toSplit = toSplit + ' ' + str(row[1])

    strArr = toSplit.split(' ')

    for thisWord in strArr:
        if(len(thisWord) > 2):
            newWord = model.MotsCle()
            newWord.motcle = thisWord.strip()
            stmt = model.session.execute(f"SELECT 1 FROM mots_cles WHERE motcle = '{newWord.motcle}' group by motcle;")
            if(not stmt.fetchone()):
                model.session.add(newWord)
                model.session.commit()
                model.session.refresh(newWord)

                stmt = model.session.execute(f"INSERT INTO public.motcle_inspection(idinspection, idmotcle)	VALUES ({idinspection}, {newWord.idmotcle});")
                model.session.commit()
            else:
                idMotCle = model.session.execute(f"SELECT idmotcle FROM mots_cles WHERE motcle = '{newWord.motcle}';").fetchone()['idmotcle']
                stmt = model.session.execute(f"SELECT 1 FROM motcle_inspection WHERE idinspection = {idinspection} and idmotcle={idMotCle}")
                if(not stmt.fetchone()):
                    stmt = model.session.execute(f"INSERT INTO public.motcle_inspection(idinspection, idmotcle)	VALUES ({idinspection}, {idMotCle});")
                    model.session.commit()



model.session.commit()


# stmt = model.session.execute("UPDATE etablissement set nb_agrements=0 where nb_agrements is null")
# model.session.commit()

In [ ]:
model.session.flush()